___
## Módulo 8: Temas avanzados.
___

## 8.2.  Web Scraping I

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

**8.2.1** Captura los hechos relevantes de la CNMV: http://www.cnmv.es/portal/HR/HRAldia.aspx

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'http://www.cnmv.es/portal/HR/HRAldia.aspx'
re = requests.get(url,headers=headers)
soup = BeautifulSoup( re.text, 'html.parser')
datos_a_extraer = soup.find_all('article',{'class' : 'block-content-pdf block-content'})
datos_a_extraer = datos_a_extraer[0]

In [ ]:
col_hora = []
horas = datos_a_extraer.find_all('li',{'class' : 'time'})
for hora in horas:    
    texto = hora.text
    texto = texto.replace('\r\n',"")
    #print(texto)
    col_hora.append(texto)

col_titulos = []
titulos = datos_a_extraer.find_all('span',{'class' : 'tit-small'})
for titulo in titulos:    
    texto = titulo.text
    #print(texto)
    col_titulos.append(texto)

col_tipo = []
tipologias = datos_a_extraer.find_all('span',{'class' : 'negrita'})
for tipo in tipologias:    
    texto = tipo.text
    #print(texto)
    col_tipo.append(texto)

col_contenido = []
contenidos = datos_a_extraer.find_all('li',{'class' : 'padding-r resumen'})
for contenido in contenidos:    
    texto = contenido.text
    texto = texto.replace('\r\n',"")
    #print(texto)
    col_contenido.append(texto)

data = {'Hora': col_hora,
        'Títulos': col_titulos,
        'Tipo': col_tipo,
        'Contenido':col_contenido}
resultado = pd.DataFrame(data)
resultado

**8.2.2** Captura las noticias sobre economía de Expansion: http://www.expansion.com/economia.html?intcmp=MENUHOM24101&s_kw=economia

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'http://www.expansion.com/economia.html?intcmp=MENUHOM24101&s_kw=economia'

In [ ]:
re = requests.get(url,headers=headers)

In [ ]:
soup = BeautifulSoup( re.text, 'html.parser')

In [ ]:
datos_a_extraer = soup.find_all('a',{'class' : 'ue-c-cover-content__link'})

In [ ]:
rows = []

for info in datos_a_extraer:
    texto = info.text
    # print(info.text)
    rows.append(texto)

In [ ]:
df = pd.DataFrame(rows)
df.columns = ['Noticias']
df

**8.2.3** Captura la agenda macroeconómica de investing.com: http://es.investing.com/economic-calendar/

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'https://es.investing.com/economic-calendar/'

In [ ]:
re = requests.get(url,headers=headers)
soup = BeautifulSoup( re.text, 'html.parser')

In [ ]:
datos_a_extraer = soup.find_all('table',{'class' : 'genTbl closedTbl ecoCalTbl persistArea js-economic-table'})
datos_a_extraer = datos_a_extraer[0]

In [ ]:
trs = datos_a_extraer.find_all('tr')

df = pd.DataFrame(columns=['Hora', 'Evento', 'Actual', 'Previsión', 'Anterior'])

for tr in trs:
    
    tds = tr.find_all('td')
    
    hora = ""
    evento = ""
    actual = ""
    prevision = ""
    anterior = ""
    
    for td in tds:
        
        if "js-time" in str(td):
            hora = td.text
            #print(texto)
        
        if "left event" in str(td):
            evento = td.text
            evento = evento.replace('\n',"")
            #print(texto)
        
        if "actual" in str(td):
            actual = td.text
            #print(texto)
            
        if "forecast" in str(td):
            prevision = td.text
            #print(texto)
        
        if "previous" in str(td):
            anterior = td.text
            #print(texto)
    
    if hora == "":
        continue
    
    df = df.append({'Hora': hora, 'Evento': evento, 'Actual': actual, 'Previsión': prevision, 'Anterior': anterior}, ignore_index=True)


In [ ]:
df

**8.2.4** Captura las cotizaciones de los commodities de investing.com: https://es.investing.com/commodities/

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'https://es.investing.com/commodities/'
re = requests.get(url,headers=headers)
soup = BeautifulSoup( re.text, 'html.parser')

In [ ]:
datos_a_extraer = soup.find_all('table',{'class' : 'genTbl closedTbl crossRatesTbl'})
datos_a_extraer = datos_a_extraer[0]

In [ ]:
trs = datos_a_extraer.find_all('tr')

df = pd.DataFrame("", index=np.arange(len(trs)), columns=['Material primas', '15 minutos', '1 hora', 'Diario', 'Semanal', 'Mensual', 'Anual', '3 años'])

fila = 0

for tr in trs:
    
    tds = tr.find_all('td') 
    
    columna = 0
    
    for td in tds:
        
        if "flag" in str(td):
            continue
            
        texto = td.text
        # print(texto)
        df.iloc[fila, columna] = texto
        
        columna = columna + 1
    
    fila = fila + 1
    

In [ ]:
df = df.iloc[1:,:]
df

**8.2.5** Captura las cotizaciones del Ibex 35 de Bolsa de Madrid: http://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'http://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'
re = requests.get(url,headers=headers)
soup = BeautifulSoup( re.text, 'html.parser')

In [ ]:
datos_a_extraer = soup.find_all('table',{'class' : 'TblPort'})
datos_a_extraer = datos_a_extraer[1]

In [ ]:
trs = datos_a_extraer.find_all('tr')

df = pd.DataFrame("", index=np.arange(len(trs)), columns=['Nombre', 'Ult', '% Dif', 'Max', 'Min', 'Volumen', 'Efectivo', 'Fecha', 'Hora'])

fila = 0

for tr in trs:
    
    tds = tr.find_all('td') 
    
    columna = 0 

    for td in tds:
        
        texto = td.text
        # print(texto)
        df.iloc[fila, columna] = texto
        
        columna = columna + 1
    
    fila = fila + 1

In [ ]:
df = df.iloc[1:,:]
df

**8.2.6** De la página de bolsa de madrid: https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx obten un daframe con los contenidos de la tabla.

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx'

In [ ]:
re = requests.get(url,headers=headers)

soup = BeautifulSoup( re.text, 'html.parser')
table = soup.find_all('table', {'class' : 'TblPort'})
table = table[0]

trs = table.find_all('tr')
ths = table.find_all('th')

columns = []
for th in ths:
    #print(th.string)
    columns.append(th.string)

rows = []
for tr in trs:
    #print(tr)
    #print(" ")
    
    tds = tr.find_all('td')
    #print()
    #print(" ")
    
    data_row = []
    for td in tds:
        #print(td.string)
        #print(" ")
        data_row.append(td.string)
    rows.append(data_row)

In [ ]:
df = pd.DataFrame(rows[1:], columns=columns)
df = df.set_index('Nombre')

In [ ]:
df.head()

**8.2.7** Obtén las últimas noticias (expansión) y precios objetivo (expansión) de: Telefónica, Santander y Siemens Gamesa

Noticias: 

- https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html
- https://www.expansion.com/mercados/cotizaciones/valores/santander_M.SAN.html
- https://www.expansion.com/mercados/cotizaciones/valores/siemensgamesa_M.SGRE.html

Precios objetivo: 

- https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/t/telefonica_M.TEF.html
- https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/s/santander_M.SAN.html
- https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/s/siemensgamesa_M.SGRE.html

In [ ]:
def ultimas_noticias(url):

    headers = {'User-Agent': 'Firefox'}
    # url = 'https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html'
    re = requests.get(url,headers=headers)
    soup = BeautifulSoup( re.text, 'html.parser')

    datos_a_extraer = soup.find_all('section',{'class' : 'ultimas_noticias_sobre dos-columnas'})
    datos_a_extraer = datos_a_extraer[0]
    datos_a_extraer = datos_a_extraer.find_all('article',{'class' : 'noticia'})
    
    # Las noticias siempre son 4, de ahí np.arange(4)
    df_noticias = pd.DataFrame("", index=np.arange(4), columns=['Fecha', 'Titulo', 'Entradilla'])

    fila = 0

    for noticia in datos_a_extraer:

        fechas = noticia.find_all('p',{'class' : 'fecha'})    
        for fecha in fechas:        
            texto = fecha.text
            #print(texto)
            df_noticias.iloc[fila,0] = texto

        titulos = noticia.find_all('h3')    
        for titulo in titulos:        
            texto = titulo.text
            texto = texto.replace('\n',"")
            #print(texto)
            df_noticias.iloc[fila,1] = texto

        entradillas = noticia.find_all('div',{'class' : 'entradilla'})    
        for entradilla in entradillas:        
            texto = entradilla.text
            texto = texto.replace('\n',"")
            #print(texto)
            df_noticias.iloc[fila,2] = texto

        fila = fila + 1       

    return df_noticias

In [ ]:
def precios_objetivos(url):

    headers = {'User-Agent': 'Firefox'}
    # url = 'https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/t/telefonica_M.TEF.html'
    re = requests.get(url,headers=headers)
    soup = BeautifulSoup( re.text, 'html.parser')

    datos_a_extraer = soup.find_all('section',{'class' : 'consenso_mercados'})
    datos_a_extraer = datos_a_extraer[0]

    datos_a_extraer = datos_a_extraer.find_all('tbody')
    datos_a_extraer = datos_a_extraer[0]

    trs = datos_a_extraer.find_all('tr')

    df_precios_objetivos = pd.DataFrame("", index=np.arange(len(trs)), columns=['Firma', 'Fecha', 'Recomendación', 'Precio objetivo', 'Revaloración potencial'])

    fila = 0

    for tr in trs:

        tds = tr.find_all('td')

        columna = 0

        for td in tds:

            texto = td.text
            #print(texto)
            df_precios_objetivos.iloc[fila, columna] = texto

            columna = columna + 1

        fila = fila + 1

    return df_precios_objetivos

In [ ]:
empresas = {
    'Telefonica': ['https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html',
                  'https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/t/telefonica_M.TEF.html'], 
    'Santander': ['https://www.expansion.com/mercados/cotizaciones/valores/santander_M.SAN.html',
                 'https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/s/santander_M.SAN.html'], 
    'Siemens Gamesa': ['https://www.expansion.com/mercados/cotizaciones/valores/siemensgamesa_M.SGRE.html',
                      'https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/s/siemensgamesa_M.SGRE.html'],
}

df_ultimas_noticias = ultimas_noticias(empresas['Telefonica'][0])
df_precios_objetivos = precios_objetivos(empresas['Telefonica'][1])

In [ ]:
df_ultimas_noticias

In [ ]:
df_precios_objetivos

**8.2.8** Desarrolla una función que sea capaz de capturar todas las noticias que tengan relación con una cualquier empresa y que hayan tenido lugar en un intervalo determinado. Utiliza para ello la hemeroteca de ABC. Pueba a obtener todas la noticias de Telefónica entre el 1 de enero y 31 de enero de 2016.

- http://www.abc.es/hemeroteca/resultados-busqueda-avanzada/noticia/pagina-1?exa=telefonica&rfec=20160101;20161031&or=1&nres=20

In [ ]:
def noticias_relacionadas(empresa, fecha_inicio, fecha_fin, pagina = 1):

    #empresa = 'Telefonica'
    empresa = empresa.lower()
    #pagina = 1

    #fecha_inicio = '1/1/2016'
    #fecha_fin = '31/12/2016'
    fechas = pd.to_datetime([fecha_inicio, fecha_fin])
    
    año_inicio = fechas[0].year
    mes_inicio = '{:02d}'.format(fechas[0].month)
    dia_inicio = '{:02d}'.format(fechas[0].day)

    año_fin = fechas[1].year
    mes_fin = '{:02d}'.format(fechas[1].month)
    dia_fin = '{:02d}'.format(fechas[1].day)

    # url = 'http://www.abc.es/hemeroteca/resultados-busqueda-avanzada/noticia/pagina-1?exa=telefonica&rfec=20160101;20161031&or=1&nres=20'
    # Ten cuidado de no tabular la url cortada, o meterás una tabulación en la url y no serás capaz de leer su contenido
    url = f'http://www.abc.es/hemeroteca/resultados-busqueda-avanzada/noticia/pagina-{pagina}?exa={empresa}&rfec={año_inicio}\
{mes_inicio}{dia_inicio};{año_fin}{mes_fin}{dia_fin}&or=1&nres=20'
    #print(url)

    headers = {'User-Agent': 'Firefox'}
    re = requests.get(url,headers=headers)
    soup = BeautifulSoup( re.text, 'html.parser')

    # Lo primero que tenemos que saber es cuantas páginas tendremos que recorrer para obtener todas las noticias
    numero_noticias = soup.find('span', {'class': 'total'})
    numero_noticias = numero_noticias.text
    numero_noticias = numero_noticias.replace('(','').replace(')','')
    numero_noticias = int(numero_noticias)
    numero_paginas = int(numero_noticias/20)
    
    noticias = []
    
    for pagina in range(1,numero_paginas+1):
    
        # Construimos la url de cada página
        url = f'http://www.abc.es/hemeroteca/resultados-busqueda-avanzada/noticia/pagina-{pagina}?exa={empresa}&rfec={año_inicio}\
{mes_inicio}{dia_inicio};{año_fin}{mes_fin}{dia_fin}&or=1&nres=20'
        
        headers = {'User-Agent': 'Firefox'}
        re = requests.get(url,headers=headers)
        soup = BeautifulSoup( re.text, 'html.parser')
    
        datos_a_extraer = soup.find_all('div',{'class' : 'clearfix'})

        noticias_pagina = pd.DataFrame("", index=np.arange(20), columns=['Fecha', 'Título', 'Contenido'])

        for noticia in datos_a_extraer:

            fechas = noticia.find_all('span',{'class' : 'date'})
            fila = 0
            for fecha in fechas:        
                texto = fecha.text
                #print(texto)
                noticias_pagina.iloc[fila, 0] = texto
                fila = fila + 1

            titulos = noticia.find_all('h2') 
            fila = 0
            for titulo in titulos:        
                texto = titulo.text
                texto = texto.replace('\n',"")
                #print(texto)
                noticias_pagina.iloc[fila, 1] = texto
                fila = fila + 1

            contenidos = noticia.find_all('p')
            fila = 0
            for contenido in contenidos:        
                texto = contenido.text
                texto = texto.replace('\n',"")
                texto = texto.replace('\r\n',"")
                #print(texto)
                noticias_pagina.iloc[fila, 2] = texto
                fila = fila + 1
    
        noticias.append(noticias_pagina)
        resultado = pd.concat(noticias)    
    
    return resultado

In [ ]:
empresa = 'Telefonica'
fecha_inicio = '1/1/2016'
fecha_fin = '31/1/2016'

noticias = noticias_relacionadas(empresa = empresa, fecha_inicio = fecha_inicio, fecha_fin = fecha_fin)
noticias

##  8.3. Web Scraping II.
**8.3.1** Cambia en el selector a Ibex Small Cap y obten la tabla de los datos en un dataframe.

In [ ]:
from selenium import webdriver

In [ ]:
DRIVER_PATH = 'chromedriver' # El mío está en la carpeta del módulo 8
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://www.bolsamadrid.es/')

boton_acciones = driver.find_element_by_xpath('//*[@id="MenuIzq"]/div[1]/div[4]/div[1]/div/a')
boton_acciones.click()

In [ ]:
botton_small_cap = driver.find_element_by_xpath('//*[@id="SelIndice"]/option[4]')

In [ ]:
botton_small_cap.text

In [ ]:
botton_small_cap.click()

In [ ]:
botton_consultar = driver.find_element_by_xpath('//*[@id="ctl00_Contenido_Consultar"]')

In [ ]:
botton_consultar.click()

In [ ]:
tabla_small_cap = driver.find_element_by_xpath('//*[@id="aspnetForm"]/div[6]') # es el path del div que está encima de table

In [ ]:
tabla_html = tabla_small_cap.get_attribute('innerHTML')

In [ ]:
df = pd.read_html(tabla_html)

In [ ]:
df[0]

**8.3.2** Obtén de Investing.com la información histórica del Banco Santander, desde el 1 de enero del 2018, hasta el 31 de diciembre del mismo año

https://es.investing.com/equities/banco-santander-historical-data

In [ ]:
DRIVER_PATH = 'chromedriver' # El mío está en la carpeta del módulo 8
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://es.investing.com/equities/banco-santander-historical-data')

Ojo, porque al acceder a investing desde código, lo primero que nos encontraremos serán 2 pop up

<center>
<img src="./entrada_investing.png"  alt="drawing" width="600"/>
</center>

In [ ]:
boton_aceptar = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
boton_aceptar.click()

In [ ]:
boton_cerrar = driver.find_element_by_xpath('//*[@id="PromoteSignUpPopUp"]/div[2]/i')
boton_cerrar.click()

In [ ]:
boton_acciones = driver.find_element_by_xpath('//*[@id="widgetFieldDateRange"]')
boton_acciones.click()

In [ ]:
fecha_inicio = driver.find_element_by_xpath('//*[@id="startDate"]')
fecha_inicio.clear()
fecha_inicio.send_keys('01/01/2018')

In [ ]:
fecha_fin = driver.find_element_by_xpath('//*[@id="endDate"]')
fecha_fin.clear()
fecha_fin.send_keys('31/12/2018')

In [ ]:
boton_aceptar = driver.find_element_by_xpath('//*[@id="applyBtn"]')
boton_aceptar.click()

In [ ]:
tabla = driver.find_element_by_xpath('//*[@id="results_box"]')
tabla_html = tabla.get_attribute('innerHTML')
df = pd.read_html(tabla_html)
df[0]